# Detecting the rectangle of the rice leaf
## This pybook is first of the development process

In [2]:
import tensorflow as tf
import numpy as np


In [3]:
STD_SIZE: int = 1600


def load_image(filepath: str):
    file_content = tf.io.read_file(filepath)
    image = tf.io.decode_jpeg(file_content, channels=3)  # load in RGB
    return tf.image.resize(image, [STD_SIZE, STD_SIZE])  # don't forget to uniform our image

In [4]:
import PIL

image = load_image("RiceLeafs/train/Healthy/IMG_20190419_123623.jpg")
image = tf.cast(image, tf.float32)
image = tf.expand_dims(image, 0)

sobel = tf.image.sobel_edges(image)
sobel_y = np.asarray(sobel[0, :, :, :, 0])  # sobel in y-direction
sobel_x = np.asarray(sobel[0, :, :, :, 1])  # sobel in x-direction

PIL.Image.fromarray(sobel_y[..., 0]).show()
PIL.Image.fromarray(sobel_x[..., 0]).show()

2022-01-16 20:44:45.393130: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-16 20:44:45.394504: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [ ]:
import cv2

#reading the image
image = cv2.imread("RiceLeafs/train/Healthy/IMG_20190419_123623.jpg")
image = cv2.resize(image, (1600, 1600))
edged = cv2.Canny(image, 10, 220)
#cv2.imshow("Edges", edged)
#cv2.waitKey(0)

#applying closing function
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 7))
closed = cv2.morphologyEx(edged, cv2.MORPH_CLOSE, kernel)
#cv2.imshow("Closed", closed)
#cv2.waitKey(0)

#finding_contours
(cnts, _) = cv2.findContours(closed.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

for c in cnts:
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)
    cv2.drawContours(image, [approx], -1, (0, 255, 0), 2)
cv2.imshow("Output", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [13]:
def find_longest_dicks(contours) -> list:  # distance in contours klist
    full_list = []
    for c in contours:
        peri = cv2.arcLength(c, True)
        poly = cv2.approxPolyDP(c, 0.02 * peri, True)
        for lin_i in range(len(poly))[:-1]:
            full_list.append(((poly[lin_i][0][0], poly[lin_i][0][1]), (poly[lin_i + 1][0][0], poly[lin_i + 1][0][1])))
    full_list = sorted(full_list, key=lambda x: -(x[0][0] - x[1][0]) ** 2 - (x[0][1] - x[1][1]) ** 2)
    return full_list


In [31]:
import cv2
from scipy import ndimage
import numpy as np
import PIL
from matplotlib.pyplot import imshow
% matplotlib inline

scharr = np.array([[1, 1, 1, 1, 1, 1, 1],
                   [1, 1, 1, 1, 1, 1, 1],
                   [1, 1, 0, 0, 0, 1, 1],
                   [1, 1, 0, 0, 0, 1, 1],
                   [1, 1, 0, 0, 0, 1, 1],
                   [1, 1, 1, 1, 1, 1, 1],
                   [1, 1, 1, 1, 1, 1, 1]])

sharpen = np.array([[0, 0, 0, -1, 0, 0, 0],
                    [0, 0, -1, -1, -1, 0, 0],
                    [0, -1, -1, -1, -1, -1, 0],
                    [-1, -1, -1, 24, -1, -1, -1],
                    [0, -1, -1, -1, -1, -1, 0],
                    [0, 0, -1, -1, -1, 0, 0],
                    [0, 0, 0, -1, 0, 0, 0]])

new_conv = np.array([[0, -1, 0],
                     [-1, 5, -1],
                     [0, -1, 0]])

image = cv2.imread("RiceLeafs/train/Healthy/IMG_20190419_123646.jpg")
image = cv2.resize(image, (1600, 1600))
no_green_image = image.copy()
no_green_image[:, :, 1] = 0
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

grad_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=21)
grad_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=21)

abs_grad_x = cv2.convertScaleAbs(grad_x)
abs_grad_y = cv2.convertScaleAbs(grad_y)

grad = np.sqrt(grad_x ** 2 + grad_y ** 2)
grad_norm = (grad * 255 / grad.max()).astype(np.uint8)
grad_zero = grad_norm.copy()
grad_zero[np.abs(grad_zero) < 64] = 0
grad_zero = grad_zero.astype(np.float)
grad_cnn = ndimage.convolve(grad_zero, sharpen, mode='constant', cval=0.0)
grad_cnn[grad_cnn > 255] = 255
grad_cnn[grad_cnn < 64] = 0
grad_cnn = grad_cnn.astype(np.uint8)
print(grad_cnn.min(), grad_cnn.max())

kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 7))
closed = cv2.morphologyEx(grad_cnn, cv2.MORPH_CLOSE, kernel)

(cnts, _) = cv2.findContours(closed.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

for c in cnts:
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)
    #print(approx, len(approx))
    cv2.drawContours(image, [approx], -1, (0, 255, 0), 2)

longest = find_longest_dicks(cnts)
for line in longest[:2]:
    cv2.line(image, line[0], line[1], color=(255, 0, 0), thickness=2)

#imshow(cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
#imshow(image)

#grad = cv2.addWeighted(abs_grad_x, 0.5, abs_grad_y, 0.5, 0)

#PIL.Image.fromarray(grad_x  ).show()
#PIL.Image.fromarray(grad_y  ).show()
#PIL.Image.fromarray(grad_norm  ).show()
#PIL.Image.fromarray(closed  ).show()
PIL.Image.fromarray(image).show()
#PIL.Image.fromarray(grad_zero  ).show()
#PIL.Image.fromarray(np.absolute(grad_cnn)  ).show()
#cv2.imshow('grad X',grad_x)
#cv2.imshow('grad Y',grad_y)
#cv2.imshow('Sobel Image',grad)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

/tmp/ipykernel_6171/2810790340.py:44: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  grad_zero = grad_zero.astype(np.float)


0 255


In [47]:
import cv2
import PIL
from scipy import ndimage
import numpy as np
import os
from math import sqrt, atan, pi


def find_longest_dicks(contours) -> list:  # distance in contours klist
    full_list = []
    for c in contours:
        peri = cv2.arcLength(c, True)
        poly = cv2.approxPolyDP(c, 0.02 * peri, True)
        for lin_i in range(len(poly))[:-1]:
            full_list.append(((poly[lin_i][0][0], poly[lin_i][0][1]), (poly[lin_i + 1][0][0], poly[lin_i + 1][0][1])))
    full_list = sorted(full_list, key=lambda x: -(x[0][0] - x[1][0]) ** 2 - (x[0][1] - x[1][1]) ** 2)
    return full_list


# k = (y1 - y2) / (x1 - x2)
# b = y2 - k*x2

def line_koeff(line) -> tuple[float, float]:
    k = (line[0][1] - line[1][1]) / (line[0][0] - line[1][0])
    b = line[1][1] - k * line[1][0]
    return k, b


def angle_diff(lhs, rhs) -> float:
    k_l, _ = line_koeff(lhs)
    k_r, _ = line_koeff(rhs)
    return abs(atan(k_l) - atan(k_r))


sharpen_cnn = np.array([[0, 0, 0, -1, 0, 0, 0],
                        [0, 0, -1, -1, -1, 0, 0],
                        [0, -1, -1, -1, -1, -1, 0],
                        [-1, -1, -1, 24, -1, -1, -1],
                        [0, -1, -1, -1, -1, -1, 0],
                        [0, 0, -1, -1, -1, 0, 0],
                        [0, 0, 0, -1, 0, 0, 0]])

line_len_f = lambda x: sqrt((x[0][0] - x[1][0]) ** 2 + (x[0][1] - x[1][1]) ** 2)
len_threshold_1 = 0.40
len_threshold_other = 0.8
angle_diff_threshold = 10 * (pi / 180)

# TODO compare results with and without shadow removal
def shadow_remove(img):
    rgb_planes = cv2.split(img)
    result_norm_planes = []
    for plane in rgb_planes:
        dilated_img = cv2.dilate(plane, np.ones((7, 7), np.uint8))
        bg_img = cv2.medianBlur(dilated_img, 21)
        diff_img = 255 - cv2.absdiff(plane, bg_img)
        norm_img = cv2.normalize(diff_img, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
        result_norm_planes.append(norm_img)
    shadowremov = cv2.merge(result_norm_planes)
    return shadowremov

def find_candidate_lines(image_data) -> list:
    #gray = cv2.cvtColor(shadow_remove(image_data), cv2.COLOR_BGR2GRAY)
    gray = cv2.cvtColor(image_data, cv2.COLOR_BGR2GRAY) # TODO cmp

    grad_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=21)
    grad_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=21)

    grad = np.sqrt(grad_x ** 2 + grad_y ** 2)
    grad_norm = (grad * 255 / grad.max()).astype(np.uint8)
    grad_zero = grad_norm.copy()
    grad_zero[np.abs(grad_zero) < 64] = 0
    grad_zero = grad_zero.astype(np.float)
    grad_cnn = ndimage.convolve(grad_zero, sharpen, mode='constant', cval=0.0)
    grad_cnn[grad_cnn > 255] = 255
    grad_cnn[grad_cnn < 64] = 0
    grad_cnn = grad_cnn.astype(np.uint8)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 7))
    closed = cv2.morphologyEx(grad_cnn, cv2.MORPH_CLOSE, kernel)

    (cnts, _) = cv2.findContours(closed.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    all_dicks = find_longest_dicks(cnts)
    longest = all_dicks[:min(10, len(all_dicks))]

    # here we filter dicks that are not big enough
    if len(longest) <= 2:
        return longest
    target = list()
    target.append(longest[0])  # at least 1 should be here
    current_thr = len_threshold_1
    for line in longest[1:]:
        prev_len = line_len_f(target[-1])
        current_len = line_len_f(line)
        if current_len < prev_len * current_thr:
            break
        if angle_diff(target[0], line) > angle_diff_threshold:
            continue
        target.append(line)
        current_thr = len_threshold_other

    return target


def show_draw_lines(path: str):
    image = cv2.imread(path)
    image = cv2.resize(image, (1600, 1600))
    lines = find_candidate_lines(image)
    for line in lines:
        cv2.line(image, line[0], line[1], color=(255, 0, 0), thickness=2)

    PIL.Image.fromarray(image).show()


def process_folder_draw_lines(folder: str, target_folder: str):
    files = os.listdir(folder)
    for filename in files[:100]:
        image = cv2.imread(os.path.join(folder, filename))
        image = cv2.resize(image, (1600, 1600))
        lines = find_candidate_lines(image)
        for line in lines:
            cv2.line(image, line[0], line[1], color=(0, 0, 255), thickness=2)
        cv2.imwrite(os.path.join(target_folder, filename), image)


#show_draw_lines("RiceLeafs/train/Healthy/IMG_20190419_123646.jpg")
process_folder_draw_lines("RiceLeafs/train/Healthy", "test")

/tmp/ipykernel_6171/1930751533.py:70: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  grad_zero = grad_zero.astype(np.float)
